In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
system_name = "Schipper Power- Three"
location = "Netherlands"
base_url = "https://pvoutput.org/list.jsp?p={}&id=49186&sid=67510&gs=0&v=0&o=date&d=desc"
pages_to_scrape = 5

In [3]:
options = Options()
options.binary_location = r"C:\Program Files\chrome for testing\chrome-win64\chrome.exe"
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
service = Service(executable_path=r"C:\Program Files\chrome for testing\chrome-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [4]:
all_data = []

for page in range(pages_to_scrape):
    url = base_url.format(page)
    print(f"📄 Scraping page {page + 1} from: {url}")
    
    try:
        driver.get(url)
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", id="tbl_main")

        if not table:
            print(f"⚠️ No table found on page {page + 1}. Retrying once...")
            time.sleep(5)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", id="tbl_main")

        if table:
            rows = table.find_all("tr")[1:]
            for row in rows:
                cols = [td.get_text(strip=True).replace(',', '') for td in row.find_all("td")]
                if len(cols) >= 9:
                    cols.append(system_name)
                    cols.append(location)
                    all_data.append(cols)
        else:
            print(f"❌ Failed again: No table found on page {page + 1}")
    except Exception as e:
        print(f"🚨 Error scraping page {page + 1}: {e}")

driver.quit()

📄 Scraping page 1 from: https://pvoutput.org/list.jsp?p=0&id=49186&sid=67510&gs=0&v=0&o=date&d=desc
📄 Scraping page 2 from: https://pvoutput.org/list.jsp?p=1&id=49186&sid=67510&gs=0&v=0&o=date&d=desc
📄 Scraping page 3 from: https://pvoutput.org/list.jsp?p=2&id=49186&sid=67510&gs=0&v=0&o=date&d=desc
📄 Scraping page 4 from: https://pvoutput.org/list.jsp?p=3&id=49186&sid=67510&gs=0&v=0&o=date&d=desc
📄 Scraping page 5 from: https://pvoutput.org/list.jsp?p=4&id=49186&sid=67510&gs=0&v=0&o=date&d=desc


In [5]:
header = [
    "Date", "Generated", "Efficiency", "Exported", "Peak Power",
    "Peak Time", "Conditions", "Temperature", "Comments",
    "System Name", "Location"
]

In [6]:
df = pd.DataFrame(all_data, columns=header)
df.drop(columns=["Exported", "Temperature", "Comments", "Peak Power", "Peak Time", "Conditions"], inplace=True)
print(df.head())

       Date    Generated   Efficiency            System Name     Location
0  30/03/25   952.360kWh  1.911kWh/kW  Schipper Power- Three  Netherlands
1  29/03/25  1415.110kWh  2.839kWh/kW  Schipper Power- Three  Netherlands
2  28/03/25   867.820kWh  1.741kWh/kW  Schipper Power- Three  Netherlands
3  27/03/25  1727.360kWh  3.466kWh/kW  Schipper Power- Three  Netherlands
4  26/03/25  1034.490kWh  2.076kWh/kW  Schipper Power- Three  Netherlands


In [20]:
page6 = pd.read_csv("Schipper Power-Three_page6.csv")
page7 = pd.read_csv("Schipper Power-Three_page7.csv")
page8 = pd.read_csv("Schipper Power-Three_page8.csv")
page9 = pd.read_csv("Schipper Power-Three_page9.csv")
page10 = pd.read_csv("Schipper Power-Three_page10.csv")

In [22]:
page6["System Name"] = "Schipper Power- Three"
page6["Location"] = "Netherlands"
page7["System Name"] = "Schipper Power- Three"
page7["Location"] = "Netherlands"
page8["System Name"] = "Schipper Power- Three"
page8["Location"] = "Netherlands"
page9["System Name"] = "Schipper Power- Three"
page9["Location"] = "Netherlands"
page10["System Name"] = "Schipper Power- Three"
page10["Location"] = "Netherlands"

In [24]:
master_df = pd.concat([df, page6, page7, page8, page9, page10], ignore_index=True)

In [26]:
master_df

,Date,Generated,Efficiency,System Name,Location
0,30/03/25,952.360kWh,1.911kWh/kW,Schipper Power- Three,Netherlands
1,29/03/25,1415.110kWh,2.839kWh/kW,Schipper Power- Three,Netherlands
2,28/03/25,867.820kWh,1.741kWh/kW,Schipper Power- Three,Netherlands
3,27/03/25,1727.360kWh,3.466kWh/kW,Schipper Power- Three,Netherlands
4,26/03/25,1034.490kWh,2.076kWh/kW,Schipper Power- Three,Netherlands
...,...,...,...,...,...
395,2/29/2024,218.1,0.437583,Schipper Power- Three,Netherlands
396,2/28/2024,580.13,1.163938,Schipper Power- Three,Netherlands
397,2/27/2024,1021.18,2.048834,Schipper Power- Three,Netherlands
398,2/26/2024,428.96,0.86064,Schipper Power- Three,Netherlands


In [28]:
master_df["System Size (kW)"] = 498.420

In [32]:
master_df.to_csv("daily_Schipper Power- Three_master.csv", index=False)